# Homework Assignment 4: Q&A System

### André Schweizer

### Import libraries

In [423]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
import re
from nltk.corpus import stopwords
from nltk import pos_tag
import string
import numpy as np
import glob
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import spacy

### Import and clean file with questions

In [1175]:
os.chdir('/Users/andreschweizer/Documents/2019W/IEMS 308/Q&A System')

with open('Questions.txt', 'r') as f:
    questions = f.read()

In [1176]:
questions = questions.split('\n')

In [1177]:
questions = filter(None,questions)

### Determine question type

In [1181]:
# Create list to insert question types
qtype = list(pd.Series(0, index = range(0,len(questions))))

In [1182]:
# Create dataframe to hold the questions and their respective types
qs = pd.DataFrame(questions, columns = ['Question']).join(pd.DataFrame(qtype, columns = ['Type']))

In [1183]:
# Determine the type of question based on the first word
count = -1
for q in qs.Question:
    count = count + 1
    sent_tokens = word_tokenize(q)
    first_word = sent_tokens[0]
    if re.search(r'[Ww]ho',first_word) != None:
        qs.Type[count] = ['PERSON']
    if re.search(r'[Ww]hen',first_word) != None:
        qs.Type[count] = ['EVENT', 'DATE','TIME','CARDINAL']
    if re.search(r'[Ww]here',first_word) != None:
        qs.Type[count] = ['FAC','ORG','GPE','LOC','EVENT','WORK_OF_ART','LAW']
    if re.search(r'[Ww]hat',first_word) != None:
        qs.Type[count] = ['ORG','PRODUCT','EVENT','LAW','PERCENT','MONEY','QUANTITY','CARDINAL']
    if re.search(r'[Ww]hich',first_word) != None:
        qs.Type[count] = ['NORP','ORG']

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [1184]:
qs

,Question,Type
0,Which companies went bankrupt in December 2017?,"[NORP, ORG]"
1,Which companies went bankrupt in July 2009?,"[NORP, ORG]"
2,Which companies went bankrupt in August 2011?,"[NORP, ORG]"
3,What affects GDP?,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA..."
4,What percentage of drop or increase is associa...,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA..."
5,Who is the CEO of Apple?,[PERSON]
6,Who is the CEO of Berkshire Hathaway?,[PERSON]
7,Who is the CEO of Amazon?,[PERSON]


### Extract keywords

In [1185]:
# Create column to store filtered questions
qs['FilteredQuestion'] = 0
qs

,Question,Type,FilteredQuestion
0,Which companies went bankrupt in December 2017?,"[NORP, ORG]",0
1,Which companies went bankrupt in July 2009?,"[NORP, ORG]",0
2,Which companies went bankrupt in August 2011?,"[NORP, ORG]",0
3,What affects GDP?,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA...",0
4,What percentage of drop or increase is associa...,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA...",0
5,Who is the CEO of Apple?,[PERSON],0
6,Who is the CEO of Berkshire Hathaway?,[PERSON],0
7,Who is the CEO of Amazon?,[PERSON],0


In [1186]:
# Remove punctuation
count = -1
for q in qs.Question:
    count = count + 1
    s = q.translate(None, string.punctuation)
    qs.FilteredQuestion[count] = s
qs

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Question,Type,FilteredQuestion
0,Which companies went bankrupt in December 2017?,"[NORP, ORG]",Which companies went bankrupt in December 2017
1,Which companies went bankrupt in July 2009?,"[NORP, ORG]",Which companies went bankrupt in July 2009
2,Which companies went bankrupt in August 2011?,"[NORP, ORG]",Which companies went bankrupt in August 2011
3,What affects GDP?,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA...",What affects GDP
4,What percentage of drop or increase is associa...,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA...",What percentage of drop or increase is associa...
5,Who is the CEO of Apple?,[PERSON],Who is the CEO of Apple
6,Who is the CEO of Berkshire Hathaway?,[PERSON],Who is the CEO of Berkshire Hathaway
7,Who is the CEO of Amazon?,[PERSON],Who is the CEO of Amazon


In [1187]:
# Remove stopwords from questions
count = -1
for q in qs.FilteredQuestion:
    count = count + 1
    sent_tokens = word_tokenize(q)
    filtered_words = [word for word in sent_tokens if word not in stopwords.words("english")]
    filtered_sent = ' '.join(filtered_words)
    qs.FilteredQuestion[count] = filtered_sent

In [1188]:
qs

,Question,Type,FilteredQuestion
0,Which companies went bankrupt in December 2017?,"[NORP, ORG]",Which companies went bankrupt December 2017
1,Which companies went bankrupt in July 2009?,"[NORP, ORG]",Which companies went bankrupt July 2009
2,Which companies went bankrupt in August 2011?,"[NORP, ORG]",Which companies went bankrupt August 2011
3,What affects GDP?,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA...",What affects GDP
4,What percentage of drop or increase is associa...,"[ORG, PRODUCT, EVENT, LAW, PERCENT, MONEY, QUA...",What percentage drop increase associated property
5,Who is the CEO of Apple?,[PERSON],Who CEO Apple
6,Who is the CEO of Berkshire Hathaway?,[PERSON],Who CEO Berkshire Hathaway
7,Who is the CEO of Amazon?,[PERSON],Who CEO Amazon


In [1189]:
keywords = [[] for x in xrange(len(questions))]

In [1190]:
# Find nouns
count = -1
for q in qs.FilteredQuestion:
    count = count + 1
    sent_tokens = word_tokenize(q)
    sent_tagged = pos_tag(sent_tokens)
    nouncount = -1
    nouns = []
    for word in sent_tagged:
        if (word[1] == 'NNP' or word[1] == 'NNPS' or word[1] == 'NN' or word[1] == 'NNS'):
            nouncount = nouncount + 1
            nouns.insert(nouncount,string.lower(word[0]))
    keywords[count] = nouns

In [1191]:
# Find words in quotation marks
count = -1
for q in qs.FilteredQuestion:
    count = count + 1
    match = re.search(r'\"(.+?)\"', q)
    if match != None:
        keywords[count].append(match.group())

In [1192]:
# Find dates
dates = [[] for x in xrange(len(questions))]
count = -1

for q in qs.FilteredQuestion:
    count = count + 1
    doc = nlp(unicode(q, errors = 'ignore'))
    for entity in doc.ents:
        if entity.label_ == 'DATE':
            dates[count] = dates[count] + [entity.text]

In [1193]:
keywords

[['companies', 'december'],
 ['companies', 'july'],
 ['companies', 'august'],
 ['gdp'],
 ['percentage', 'drop', 'increase', 'property'],
 ['apple'],
 ['ceo', 'berkshire', 'hathaway'],
 ['ceo', 'amazon']]

### Import corpus

In [63]:
# Create list of paths to files in corpus
docpaths = glob.glob('/Users/andreschweizer/Documents/2019W/IEMS 308/Text Analytics/Corpus/' + '[!~]*.txt')

In [212]:
# Get texts out of corpus into one list of strings with preliminary sentence segmentation.
# Each item is a sentence in the corpus separated by .,!, or ?
text = [[] for x in xrange(len(docpaths))]
count = -1

for element in docpaths:
    count = count + 1
    with open(element, 'r') as f:
        text[count] =  f.read()

### Document selection

In [67]:
### Create entries for all words in dictionary
# Split text into sentences
count = -1
text_sent = [[] for x in xrange(len(docpaths))]
text_words = [[] for x in xrange(len(docpaths))]

for doc in text:
    count = count + 1
    text_sent[count] = re.split(r'[.!?]',doc)
    text_words[count] = [[] for x in xrange(len(text_sent[count]))]

In [ ]:
# Split text into words
doccount = -1

for doc in text_sent:
    doccount = doccount + 1
    sentcount = -1
    for sent in doc:
        sentcount = sentcount + 1
        text_words[doccount][sentcount] = word_tokenize(sent)

In [98]:
# Create dictionary with words as keys and empty values
words = {}

for doc in text_words:
    for sent in doc:
        for word in sent:
            words[word] = []

In [ ]:
# Make values of dictionary equal to the docs that contain each of the words
doccount = -1

for doc in text_words:
    doccount = doccount + 1
    print(doccount)
    for sent in doc:
        for word in sent:
            words[word] = words[word] + [doccount]

In [ ]:
# Compile list of docs that contain at least one keyword from the questions
relevantdocs = [[] for x in xrange(len(questions))]
questioncount = -1

for q in keywords:
    questioncount = questioncount + 1
    for word in q:
        relevantdocs[questioncount] = list(set(relevantdocs[questioncount] + words[word]))

In [726]:
# Remove bytes that can't be understood by decoder
textcleaned = [[] for x in xrange(len(docpaths))]
count = -1

for doc in text:
    count = count + 1
    textcleaned[count] = unicode(text[count], errors = 'ignore')

In [220]:
# Compute tf-idf scores (docs, words)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textcleaned)
tfidfscores = pd.DataFrame(X.toarray())

In [228]:
tfidfscores.head()

,0,1,2,3,4,5,6,7,8,9,...,165381,165382,165383,165384,165385,165386,165387,165388,165389,165390
0,0.001797,0.012511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000565,0.014668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000605,0.010718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.002478,0.010978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.005429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [727]:
# Change the names of columns in matrix to be the words they represent
tfidfscores.columns = vectorizer.get_feature_names()
tfidfscores.head()

,00,000,0000,00000001,0000000545,0000001,0000009,00001,00001387015,00005,...,zyngazynga,zysman,zytiga,zyuganov,zyuzin,zyuzins,zyvlhwgwekermlk5,zywiec,zzzz,zzzzzzzzzz
0,0.001797,0.012511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000565,0.014668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000605,0.010718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.002478,0.010978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.005429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [738]:
# Calculate doc-question scores
scorelist = [[] for x in xrange(len(questions))]
questioncount = -1

for q in questions:
    questioncount = questioncount + 1
    scorelist[questioncount] = [0 for x in xrange(len(docpaths))]
    doccount = -1
    for doc in scorelist[questioncount]:
        doccount = doccount + 1
        for kw in keywords[questioncount]:
            scorelist[questioncount][doccount] = scorelist[questioncount][doccount] + tfidfscores[str(kw)][doccount]

In [739]:
# Find n docs with highest scores for each question
highestscores = [[] for x in xrange(len(questions))]
n = 20
questioncount = -1

for q in range(0,len(highestscores)):
    questioncount = questioncount + 1
    topnindex = np.argsort(scorelist[q])[-n:]
    highestscores[questioncount] = topnindex

In [740]:
highestscores

[array([452, 449, 372,  31, 373,   3, 366, 725, 408, 213, 374, 459, 364,
        158,  68, 159, 431, 387, 724, 379]),
 array([602, 594, 591, 608, 616, 246, 211, 569, 605,  68, 581, 431, 610,
        158, 159, 216, 229, 212, 577, 213]),
 array([611, 613, 243, 591, 246, 219, 627, 213, 594, 605, 431, 610, 245,
        247, 244, 640,  68, 158, 159, 248]),
 array([270, 145, 678, 540, 499, 228,  86, 721, 221, 575, 176, 484, 564,
         19, 117, 115,  29, 211, 565, 340]),
 array([240,  64, 652, 341, 527, 486, 701, 394, 176, 115, 337, 314, 352,
        149, 361, 379, 585, 180, 369, 229]),
 array([ 28, 140, 620, 318, 477,  44, 265, 239, 113, 296, 111, 616, 348,
        617, 694, 646, 112, 254,  37, 697]),
 array([439, 639, 120, 277, 590, 216, 567,   3, 123,  60, 602, 122, 660,
        426,  59, 486,  44, 508, 424, 487]),
 array([569, 527, 729, 103, 680, 662, 297, 659, 567, 678, 686, 296, 571,
        667, 532, 479, 287, 725, 679, 682])]

### Tag retrieved documents with type

In [ ]:
# Get single list of relevant documents
relevantdocs = list(set([j for i in highestscores for j in i]))

In [402]:
# Get sentences in relevant documents
relevantsentences = [[] for x in xrange(len(relevantdocs))]
doccount = -1

for doc in relevantdocs:
    doccount = doccount + 1
    relevantsentences[doccount] = text_sent[doc]

In [420]:
# Create list of sentences that are relevant to each question
sentsquestions = [[] for x in xrange(len(questions))]
questioncount = -1

for q in sentsquestions:
    questioncount = questioncount + 1
    for doc in highestscores[questioncount]:
        sentsquestions[questioncount] = sentsquestions[questioncount] + relevantsentences[relevantdocs.index(doc)]

In [425]:
# Tag sentences
nlp = spacy.load('en_core_web_sm')

In [ ]:
senttags = [[] for x in xrange(len(questions))]

for q in range(0, len(sentsquestions)):
    senttags[q] = [[] for x in xrange(len(sentsquestions[q]))]
    for sent in range(len(sentsquestions[q])):
        senttags[q][sent] = nlp(unicode(sentsquestions[q][sent], errors = 'ignore'))

### Sentence extraction

In [1195]:
# Tag keywords
questioncount = -1
for q in qs.FilteredQuestion:
    questioncount = questioncount + 1
    keywords[questioncount] = keywords[questioncount] + dates[questioncount]
keywords[0] = keywords[0] + ['bankrupt']
    
questioncount = -1

for q in keywords:
    questioncount = questioncount + 1
    kwcount = -1
    for kw in keywords[questioncount]:
        kwcount = kwcount + 1
        keywords[questioncount][kwcount] = str(kw)
        keywords[questioncount][kwcount] = unicode(keywords[questioncount][kwcount], errors = 'ignore')

In [1196]:
keywords

[[u'companies', u'december', u'December 2017', u'bankrupt'],
 [u'companies', u'july', u'July 2009'],
 [u'companies', u'august', u'August 2011'],
 [u'gdp'],
 [u'percentage', u'drop', u'increase', u'property'],
 [u'apple'],
 [u'ceo', u'berkshire', u'hathaway'],
 [u'ceo', u'amazon']]

In [ ]:
# Find sentences that contain at least one of the tags of the question type
importantsents = [[] for x in xrange(len(questions))]
questioncount = -1

for q in senttags:
    questioncount = questioncount + 1
    importantsents[questioncount] = [[] for x in xrange(len(senttags[questioncount]))]
    sentcount = -1
    for sent in q:
        sentcount = sentcount + 1
        relevant = False
        if sent.ents:
            for entity in sent.ents:
                if entity.label_ in qs.Type[questioncount]:
                    relevant = True
                    break
        importantsents[questioncount][sentcount] = relevant

In [763]:
# Only keep sentences that contain the same tag
finalsents = [[] for x in xrange(len(questions))]
questioncount = -1

for q in sentsquestions:
    questioncount = questioncount + 1
    sentcount = -1
    for sent in q:
        sentcount = sentcount + 1
        if importantsents[questioncount][sentcount]:
            finalsents[questioncount] = finalsents[questioncount] + [sentsquestions[questioncount][sentcount]]

### Score sentences

In [1197]:
# Add expressions in questions to be used as search parameters in sentences
questioncount = -1

for q in questions:
    questioncount = questioncount + 1
    r = Rake()
    r.extract_keywords_from_text(q)
    newkws = r.get_ranked_phrases()
    keywords[questioncount] = keywords[questioncount] + newkws
    
keywords[0] = keywords[0] + ['bankrupt']

In [1198]:
keywords

[[u'companies',
  u'december',
  u'December 2017',
  u'bankrupt',
  'companies went bankrupt',
  'december 2017',
  'bankrupt'],
 [u'companies', u'july', u'July 2009', 'companies went bankrupt', 'july 2009'],
 [u'companies',
  u'august',
  u'August 2011',
  'companies went bankrupt',
  'august 2011'],
 [u'gdp', 'affects gdp'],
 [u'percentage',
  u'drop',
  u'increase',
  u'property',
  'property',
  'percentage',
  'increase',
  'drop',
  'associated'],
 [u'apple', 'ceo', 'apple'],
 [u'ceo', u'berkshire', u'hathaway', 'berkshire hathaway', 'ceo'],
 [u'ceo', u'amazon', 'ceo', 'amazon']]

In [766]:
# Find number of keywords of each question present in each of the sentences
sentscores2 = [[] for x in xrange(len(questions))]
questioncount =  -1

for q in keywords:
    questioncount = questioncount + 1
    sentscores2[questioncount] = [0 for x in xrange(len(finalsents[questioncount]))]
    sentcount = -1
    for sent in finalsents[questioncount]:
        sentcount = sentcount + 1
        for kw in q:
            if kw in str.lower(str(unicode(sent, errors = 'ignore'))):
                sentscores2[questioncount][sentcount] = sentscores2[questioncount][sentcount] + 1

In [1203]:
# Find n sentences with highest number of keywords for each question
bestsents2= [[] for x in xrange(len(questions))]
n = 5
questioncount = -1

for q in range(0,len(bestsents2)):
    questioncount = questioncount + 1
    topnindex = np.argsort(sentscores2[q])[-n:]
    bestsents2[questioncount] = topnindex

In [1204]:
bestsents2

[array([ 301,  718, 6987,  715, 6931]),
 array([9022, 5196, 5708, 4862, 6298]),
 array([ 288,  317,  302,   25, 6729]),
 array([ 7926, 10121,   359,  5809,  6620]),
 array([ 3485,  3689, 12648, 11211,  2244]),
 array([1794, 3549,  751,  365, 3382]),
 array([1046,  386, 4071, 4192, 1327]),
 array([1603, 2893, 1221, 1425, 3591])]

### Extract answers from best sentences

In [1205]:
# Create answers
answers = [[] for x in range(0,len(questions))]
sentwithanswer = [[] for x in range (0,len(questions))]
questioncount = -1

# Find CEO names
for q in questions:
    questioncount = questioncount + 1
    if re.search(r'[Ww]ho', q) != None:
        match = re.search(r'CEO\s([A-Z](?:[a-zA-Z0-9])+\s[A-Z](?:[a-zA-Z0-9])+)', finalsents[questioncount][bestsents2[questioncount][0]])
        if match != None:
            answers[questioncount] = (answers[questioncount] + [match.group()[4:]])[0]
            sentwithanswer[questioncount] = bestsents2[questioncount][0]

questioncount = -1

for q in answers:
    questioncount = questioncount + 1
    if type(q) == str:
        answers[questioncount] = unicode(q, errors = 'ignore')
        

In [1206]:
answers

[[], [], [], [], [], u'Tim Cook', u'Warren Buffett', u'Jeff Bezos']

In [1207]:
# Find GDP factor
questioncount = -1

for q in questions:
    questioncount = questioncount + 1
    foundanswer = False
    if re.search(r'[AaEe]ffect.+?\sGDP', q) != None:
        for sent in bestsents2[questioncount]:
            doc = nlp(unicode(finalsents[questioncount][sent], errors = 'ignore'))
            for entity in doc.ents:
                if entity.label_ == 'ORG' and re.search(r'GDP', entity.text) == None:
                    foundanswer = True
                    answers[questioncount] = entity.text[1:]
                    sentwithanswer[questioncount] = sent
                    break
            if foundanswer:
                break

In [1208]:
answers

[[],
 [],
 [],
 u'Personal Consumption',
 [],
 u'Tim Cook',
 u'Warren Buffett',
 u'Jeff Bezos']

In [1209]:
# Find related percentage
questioncount = -1

for q in questions:
    questioncount = questioncount + 1
    if re.search(r'[AaEe]ffect.+?\sGDP', q) != None:
            match = re.search(r'([0-9]+\%)', finalsents[questioncount][sentwithanswer[questioncount]])
            if match != None:
                answers[questioncount + 1] = unicode(match.group(), errors = 'ignore')

In [1210]:
answers

[[],
 [],
 [],
 u'Personal Consumption',
 u'71%',
 u'Tim Cook',
 u'Warren Buffett',
 u'Jeff Bezos']

In [1211]:
# Bankrupt companies
questioncount = -1

for q in questions:
    questioncount = questioncount + 1
    if 'bankrupt' in q:
        for sent in bestsents2[questioncount]:
            doc = nlp(unicode(finalsents[questioncount][sent], errors = 'ignore'))
            for entity in doc.ents:
                if entity.label_ == 'ORG':
                    answers[questioncount].append(entity.text)

In [1212]:
answers

[[u' RED INK Bank of America',
  u'Merrill Lynch',
  u'Lehman Brothers Holdings Inc',
  u'Merrill',
  u' The exchange',
  u'U',
  u'Tokyo Metropolitan Police',
  u'the Russian Central Bank',
  u'RCB',
  u'Trust Bank'],
 [u' The Business Dynamics Statistics',
  u'Hathaway',
  u'Citi',
  u'CNBC',
  u'FREE AppDownload\r\n',
  u'Reuters',
  u' Time Warneris'],
 [u' US',
  u' US',
  u' US',
  u'CFA Institute',
  u'CFA Institute',
  u'the Ministry of Transport'],
 u'Personal Consumption',
 u'71%',
 u'Tim Cook',
 u'Warren Buffett',
 u'Jeff Bezos']

In [1213]:
# Remove double answers
questioncount = -1

for q in answers:
    questioncount = questioncount + 1
    if type(q) != list:
        answers[questioncount] = [q]

questioncount = -1

for q in answers:
    questioncount = questioncount + 1
    anscount = -1
    for ans in answers[questioncount]:
        anscount = anscount + 1
        if type(answers[questioncount][anscount]) != unicode:
            answers[questioncount][anscount] = unicode(answers[questioncount][anscount], errors = 'ignore')

questioncount = -1            
            
for q in answers:
    questioncount = questioncount + 1
    answers[questioncount] = list(set(answers[questioncount]))

In [1214]:
answers

[[u'Merrill Lynch',
  u'Merrill',
  u'Tokyo Metropolitan Police',
  u'Trust Bank',
  u'U',
  u'Lehman Brothers Holdings Inc',
  u' The exchange',
  u' RED INK Bank of America',
  u'the Russian Central Bank',
  u'RCB'],
 [u'FREE AppDownload\r\n',
  u'CNBC',
  u'Hathaway',
  u' Time Warneris',
  u'Citi',
  u'Reuters',
  u' The Business Dynamics Statistics'],
 [u'CFA Institute', u'the Ministry of Transport', u' US'],
 [u'Personal Consumption'],
 [u'71%'],
 [u'Tim Cook'],
 [u'Warren Buffett'],
 [u'Jeff Bezos']]

In [1243]:
bestsents= []
n = 50
questioncount = -1

topnindex = np.argsort(sentscores2[3])[-n:]
bestsents = topnindex

In [1244]:
bestsents3 = []

for sent in bestsents:
    if ('%' or 'percent') in finalsents[3][sent]:
        bestsents3.append(finalsents[3][sent])

In [1390]:
GDPanswers = [[],[]]
# Remove stopwords from new sentences
count = -1

for sent in bestsents3:
    count = count + 1
    sent_tokens = word_tokenize(sent)
    filtered_words = [word for word in sent_tokens if word not in stopwords.words("english")]
    filtered_sent = ' '.join(filtered_words)
    bestsents3[count] = filtered_sent

In [1391]:
# Take important words
foundanswer1 = False
justfoundanswer1 = False
foundanswer2 = False

for sent in bestsents3:
    wordtok = word_tokenize(sent)
    senttag = pos_tag(wordtok)
    for word in senttag:
        justfoundanswer1 = False
        if word[1] == 'VBZ' and foundanswer1 == False:
            GDPanswers[0].append([word[0]])
            match = re.search(r'[0-9].+?\%', sent)
            GDPanswers[0].append([match.group()])
            foundanswer1 = True
            justfoundanswer1 = True
        if justfoundanswer1 == False and foundanswer2 == False:
            match = re.search(r'([a-z]+\sspend[a-z]+)\s',sent)
            if match != None:
                GDPanswers[1].append([match.group()])
                match = re.search(r'[0-9].+?\%', sent)
                GDPanswers[1].append([match.group()])
                foundanswer2 = True

In [1392]:
GDPanswers

[[['imports'], ['16 %']], [['consumer spending '], ['39 %']]]

In [1393]:
newanswers = [[] for x in range(0,len(questions)+4)]

for ans in range(0,len(newanswers)):
    if ans in range(0,5):
        newanswers[ans] = answers[ans]
    if ans in range(5,7):
        newanswers[ans] = GDPanswers[0][ans-5]
    if ans in range(7,9):
        newanswers[ans] = GDPanswers[1][ans-7]
    if ans in range(9,12):
        newanswers[ans] = answers[ans-4]

In [1394]:
newanswers

[[u'Merrill Lynch',
  u'Merrill',
  u'Tokyo Metropolitan Police',
  u'Trust Bank',
  u'U',
  u'Lehman Brothers Holdings Inc',
  u' The exchange',
  u' RED INK Bank of America',
  u'the Russian Central Bank',
  u'RCB'],
 [u'FREE AppDownload\r\n',
  u'CNBC',
  u'Hathaway',
  u' Time Warneris',
  u'Citi',
  u'Reuters',
  u' The Business Dynamics Statistics'],
 [u'CFA Institute', u'the Ministry of Transport', u' US'],
 [u'Personal Consumption'],
 [u'71%'],
 ['imports'],
 ['16 %'],
 ['consumer spending '],
 ['39 %'],
 [u'Tim Cook'],
 [u'Warren Buffett'],
 [u'Jeff Bezos']]

In [1395]:
# Print answers and questions
for q in range(0,len(newanswers)):
    if q in range(0,5):
        print questions[q]
        for ans in newanswers[q]:
            print str(ans)
        print '\n'
    if q in range(5,7):
        print questions[q-2]
        print newanswers[q][0]
        print '\n'
    if q in range(7,9):
        print questions[q-4]
        print newanswers[q][0]
        print '\n'
    if q in range(9,12):
        print questions[q-4]
        print newanswers[q][0]
        print '\n'

Which companies went bankrupt in December 2017?
Merrill Lynch
Merrill
Tokyo Metropolitan Police
Trust Bank
U
Lehman Brothers Holdings Inc
 The exchange
 RED INK Bank of America
the Russian Central Bank
RCB


Which companies went bankrupt in July 2009?
FREE AppDownload

CNBC
Hathaway
 Time Warneris
Citi
Reuters
 The Business Dynamics Statistics


Which companies went bankrupt in August 2011?
CFA Institute
the Ministry of Transport
 US


What affects GDP?
Personal Consumption


What percentage of drop or increase is associated with this property?
71%


What affects GDP?
imports


What percentage of drop or increase is associated with this property?
16 %


What affects GDP?
consumer spending 


What percentage of drop or increase is associated with this property?
39 %


Who is the CEO of Apple?
Tim Cook


Who is the CEO of Berkshire Hathaway?
Warren Buffett


Who is the CEO of Amazon?
Jeff Bezos




In [1397]:
outputfile = open("SampleOutput.txt",'w')
for q in range(0,len(newanswers)):
    if q in range(0,5):
        outputfile.write(questions[q])
        outputfile.write('\n')
        for ans in newanswers[q]:
            outputfile.write(str(ans))
            outputfile.write('\n')
        outputfile.write('\n')
    if q in range(5,7):
        outputfile.write(questions[q-2])
        outputfile.write('\n')
        outputfile.write(newanswers[q][0])
        outputfile.write('\n\n')
    if q in range(7,9):
        outputfile.write(questions[q-4])
        outputfile.write('\n')
        outputfile.write(newanswers[q][0])
        outputfile.write('\n\n')
    if q in range(9,12):
        outputfile.write(questions[q-4])
        outputfile.write('\n')
        outputfile.write(newanswers[q][0])
        outputfile.write('\n\n')
        
outputfile.close()